In [19]:
## reference: https://colab.research.google.com/drive/1QhSnbh-WJVGZjQJF8u974msOL_vAgMeS#scrollTo=PlAGuj5kuZm9
## https://github.com/eisenjulian/nlp_estimator_tutorial/blob/master/nlp_estimators.ipynb

import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn import feature_selection
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVR
from sklearn.svm import SVC

from keras.preprocessing.text import Tokenizer

import tensorflow as tf
from tensorboard import summary as summary_lib
from tensorflow.python.keras.preprocessing import sequence

import seaborn as sns
import matplotlib.pyplot as plt

import tempfile
import os
print(tf.__version__)


dir='/Users/xinwang/ai/dataset/kaggle/DonorsChoose/'
train_file='train_1.csv'
eval_file='train_2.csv'
eval_file='train_2.csv'
test_file='test.csv'
resource_file='resources.csv'
SEED=1000
positive_sample_size=100 

train_df = pd.read_csv(dir + train_file)
eval_df = pd.read_csv(dir + eval_file)
test_df = pd.read_csv(dir + test_file)
resource_df = pd.read_csv(dir + resource_file)
label = LabelEncoder()
low_memory=False


def sampleData():
    train_label_1_df = train_df[train_df['project_is_approved']==1].sample(n=positive_sample_size,
                                                                           random_state=SEED)
    train_label_0_df = train_df[train_df['project_is_approved']==0]

    train_data = pd.concat([train_label_1_df,train_label_0_df])
    train_data = shuffle(train_data)
    
    return train_data

train_df.dropna(subset=["teacher_prefix"], inplace=True)
eval_df.dropna(subset=["teacher_prefix"], inplace=True)

train_data = train_df
eval_data = eval_df

train_data.head()

1.9.0


/Users/xinwang/ai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [ ]:
feature_columns = []

vocab_size=30000
embedding_size=50

teacher_id = tf.feature_column.categorical_column_with_hash_bucket('teacher_id', 100000)
feature_columns.append(teacher_id)

teacher_prefix = tf.feature_column.categorical_column_with_vocabulary_list(
    "teacher_prefix", [
        "Mrs.","Ms.","Mr.","Teacher","Dr."
    ])
feature_columns.append(teacher_prefix)

school_state = tf.feature_column.categorical_column_with_vocabulary_list(
    "school_state",[
        "AK","AL","AR","AZ","CA","CO","CT","DC","DE","FL","GA","HI","IA","ID","IL",
        "IN","KS","KY","LA","MA","MD","ME","MI","MN","MO","MS","MT","NC","ND","NE",
        "NH","NJ","NM","NV","NY","OH","OK","OR","PA","RI","SC","SD","TN",
        "TX","UT","VA","VT","WA","WI","WV","WY"
    ])
feature_columns.append(school_state)

# todo project_submitted_datetime

project_grade_category = tf.feature_column.categorical_column_with_vocabulary_list(
    "project_grade_category",["Grades 3-5","Grades 6-8","Grades 9-12","Grades PreK-2"])
feature_columns.append(project_grade_category)

project_subject_categories = tf.feature_column.categorical_column_with_vocabulary_list(
    "project_subject_categories",[
       "Applied Learning","Health & Sports","History & Civics","Literacy & Language",
        "Math & Science","Music & The Arts","Special Needs","Warmth"])
feature_columns.append(project_subject_categories)


project_subject_subcategories = tf.feature_column.categorical_column_with_vocabulary_list(
    "project_subject_subcategories",[
        "Applied Learning","Care & Hunger","Health & Sports","History & Civics",
        "Literacy & Language","Math & Science","Music & The Arts",
        "Warmth","Applied Sciences","Character Education","Civics & Government",
        "College & Career Prep","Community Service","ESL","Early Development",
        "Economics","Environmental Science","Extracurricular","Financial Literacy",
        "Foreign Languages","Gym & Fitness","Health & Life Science","Health & Wellness",
        "History & Geography","Literacy","Literature & Writing","Mathematics","Music",
        "Nutrition Education","Other","Parent Involvement","Performing Arts",
        "Social Sciences","Special Needs","Team Sports","Visual Arts"])
feature_columns.append(project_subject_subcategories)

project_title = tf.feature_column.categorical_column_with_hash_bucket(
    'project_title', 10000)
feature_columns.append(project_title)

for i in range(sentence_size):
    name = 'project_essay_1' + "_" + str(i)
    feature_columns.append(tf.feature_column.numeric_column(name))

    
project_essay_2 = tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity('project_essay_2', vocab_size),
    dimension=embedding_size)
# feature_columns.append(project_essay_2)

project_resource_summary = tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity('project_resource_summary',10000),
    dimension=embedding_size)
# feature_columns.append(project_resource_summary)

teacher_number_of_previously_posted_projects = tf.feature_column.bucketized_column(
    tf.feature_column.numeric_column('teacher_number_of_previously_posted_projects'), 
    boundaries=[2,7,20,50])
feature_columns.append(teacher_number_of_previously_posted_projects)


target = 'project_is_approved'
model_dir = tempfile.mkdtemp()

    
all_classifiers = {}
def train_and_evaluate(classifier):
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn(), steps = 10000)
    eval_results = classifier.evaluate(input_fn=eval_input_fn())
    
    predictions = np.array([p['logistic'][0] for p in classifier.predict(
        input_fn=eval_input_fn())])

    tf.reset_default_graph()
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, 
                              labels = eval_data[target].astype(bool),
                              num_thresholds = 21)
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'),
                                      sess.graph)
        writer.add_summary(sess.run(pr), global_step = 0)
        writer.close()
        


In [20]:
vocab_size=30000
sentence_size=100

pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2

def text_to_vector(feature_name, data_set):
    texts = data_set[feature_name].values
    
    tokenizer = Tokenizer(vocab_size)
    tokenizer.fit_on_texts(texts)
    essay_1_sequence = tokenizer.texts_to_sequences(texts)

    feature_name_df = pd.DataFrame(data = sequence.pad_sequences(essay_1_sequence,
                                                                 maxlen=sentence_size,
                                                                 truncating='post',
                                                                 padding='post',
                                                                 value=pad_id),
                                   columns = [ feature_name + "_" + str(k) for k in range(sentence_size)])
    return feature_name_df


FEATURES = ['teacher_id','teacher_prefix',
            'school_state','project_grade_category',
            'project_subject_categories','project_subject_subcategories',
            'project_title',
#             'project_essay_1',
#             'project_essay_2','project_resource_summary',
            'teacher_number_of_previously_posted_projects'
           ]

def get_input_fn(data_set, num_epochs=None, shuffle=True):
    feature_2_data = {k: data_set[k].values for k in FEATURES}
    
    for i in range(sentence_size):
        name = 'project_essay_1' + "_" + str(i)
        project_essay_1_df = text_to_vector('project_essay_1', data_set)
        
        feature_2_data[name] = project_essay_1_df[name].values
    
    return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame(feature_2_data),
      y=pd.Series(data_set[target].values),
      num_epochs=num_epochs,
      shuffle=shuffle)

def train_input_fn():
    return get_input_fn(train_data)

def eval_input_fn():
    return get_input_fn(eval_data, num_epochs=1,shuffle=False)


classifier = tf.estimator.LinearClassifier(feature_columns, 
                                           model_dir=os.path.join(model_dir, 'bow_sparse'))
train_and_evaluate(classifier)

,project_essay_1_0,project_essay_1_1,project_essay_1_2,project_essay_1_3,project_essay_1_4,project_essay_1_5,project_essay_1_6,project_essay_1_7,project_essay_1_8,project_essay_1_9,...,project_essay_1_90,project_essay_1_91,project_essay_1_92,project_essay_1_93,project_essay_1_94,project_essay_1_95,project_essay_1_96,project_essay_1_97,project_essay_1_98,project_essay_1_99
0,70,5,9,139,4,32,25,78,109,480,...,3,374,5,3,53,30,318,294,187,741
1,15,236,12,16,6,783,678,12,19,6,...,280,150,401,743,1356,701,0,0,0,0
2,1947,10,64,1221,16,2150,14794,13,62,364,...,15,190,13,62,157,1,33,3509,364,47
3,9,4,7,3,1182,4,50,7,975,2,...,260,19,4987,240,8,12,33,3,4,17
4,9,4,7,1251,2,4,45,7,694,8,...,0,0,0,0,0,0,0,0,0,0


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/hz/zfwx8n_d19g70bf5p8jpl4f43zxvln/T/tmpenquk4b5/bow_sparse', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x126397780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoi

INFO:tensorflow:global_step/sec: 53.0897
INFO:tensorflow:loss = 2219.4893, step = 7301 (1.884 sec)
INFO:tensorflow:global_step/sec: 49.2693
INFO:tensorflow:loss = 725.5602, step = 7401 (2.029 sec)
INFO:tensorflow:global_step/sec: 54.6074
INFO:tensorflow:loss = 1647.5571, step = 7501 (1.831 sec)
INFO:tensorflow:global_step/sec: 55.6374
INFO:tensorflow:loss = 444.88123, step = 7601 (1.797 sec)
INFO:tensorflow:global_step/sec: 51.3032
INFO:tensorflow:loss = 1590.6527, step = 7701 (1.950 sec)
INFO:tensorflow:global_step/sec: 56.2297
INFO:tensorflow:loss = 991.81335, step = 7801 (1.778 sec)
INFO:tensorflow:global_step/sec: 48.8706
INFO:tensorflow:loss = 1657.3751, step = 7901 (2.046 sec)
INFO:tensorflow:global_step/sec: 52.8749
INFO:tensorflow:loss = 773.33026, step = 8001 (1.891 sec)
INFO:tensorflow:global_step/sec: 50.0746
INFO:tensorflow:loss = 1114.0769, step = 8101 (2.001 sec)
INFO:tensorflow:global_step/sec: 51.2123
INFO:tensorflow:loss = 873.6095, step = 8201 (1.949 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 49.1799
INFO:tensorflow:loss = 2035.5892, step = 15601 (2.033 sec)
INFO:tensorflow:global_step/sec: 46.3769
INFO:tensorflow:loss = 3533.802, step = 15701 (2.156 sec)
INFO:tensorflow:global_step/sec: 53.6625
INFO:tensorflow:loss = 638.45215, step = 15801 (1.864 sec)
INFO:tensorflow:global_step/sec: 48.4925
INFO:tensorflow:loss = 2977.2651, step = 15901 (2.062 sec)
INFO:tensorflow:global_step/sec: 50.5868
INFO:tensorflow:loss = 1078.1871, step = 16001 (1.977 sec)
INFO:tensorflow:global_step/sec: 51.0646
INFO:tensorflow:loss = 972.36475, step = 16101 (1.959 sec)
INFO:tensorflow:global_step/sec: 53.6008
INFO:tensorflow:loss = 765.25806, step = 16201 (1.866 sec)
INFO:tensorflow:global_step/sec: 52.6676
INFO:tensorflow:loss = 1589.2417, step = 16301 (1.898 sec)
INFO:tensorflow:global_step/sec: 51.1077
INFO:tensorflow:loss = 866.9309, step = 16401 (1.959 sec)
INFO:tensorflow:global_step/sec: 46.9855
INFO:tensorflow:loss = 2493.8174, step = 16501 (2.128 sec)
IN

INFO:tensorflow:loss = 793.3423, step = 23801 (2.019 sec)
INFO:tensorflow:global_step/sec: 49.9541
INFO:tensorflow:loss = 1006.6572, step = 23901 (2.003 sec)
INFO:tensorflow:global_step/sec: 51.1371
INFO:tensorflow:loss = 1475.6814, step = 24001 (1.954 sec)
INFO:tensorflow:global_step/sec: 51.5373
INFO:tensorflow:loss = 2872.5874, step = 24101 (1.943 sec)
INFO:tensorflow:global_step/sec: 49.1678
INFO:tensorflow:loss = 513.9846, step = 24201 (2.031 sec)
INFO:tensorflow:global_step/sec: 56.022
INFO:tensorflow:loss = 509.81674, step = 24301 (1.789 sec)
INFO:tensorflow:global_step/sec: 48.9196
INFO:tensorflow:loss = 527.85645, step = 24401 (2.044 sec)
INFO:tensorflow:global_step/sec: 49.814
INFO:tensorflow:loss = 1040.6649, step = 24501 (2.004 sec)
INFO:tensorflow:global_step/sec: 50.9961
INFO:tensorflow:loss = 862.81366, step = 24601 (1.961 sec)
INFO:tensorflow:global_step/sec: 52.5912
INFO:tensorflow:loss = 461.04077, step = 24701 (1.901 sec)
INFO:tensorflow:global_step/sec: 49.2392
INFO

INFO:tensorflow:loss = 552.6, step = 31901 (2.141 sec)
INFO:tensorflow:global_step/sec: 46.1368
INFO:tensorflow:loss = 582.63214, step = 32001 (2.169 sec)
INFO:tensorflow:global_step/sec: 48.7188
INFO:tensorflow:loss = 528.444, step = 32101 (2.051 sec)
INFO:tensorflow:global_step/sec: 52.7958
INFO:tensorflow:loss = 413.5217, step = 32201 (1.894 sec)
INFO:tensorflow:global_step/sec: 51.0509
INFO:tensorflow:loss = 621.82263, step = 32301 (1.958 sec)
INFO:tensorflow:global_step/sec: 52.2421
INFO:tensorflow:loss = 769.3197, step = 32401 (1.914 sec)
INFO:tensorflow:global_step/sec: 52.9869
INFO:tensorflow:loss = 473.54333, step = 32501 (1.887 sec)
INFO:tensorflow:global_step/sec: 51.2531
INFO:tensorflow:loss = 306.4696, step = 32601 (1.951 sec)
INFO:tensorflow:global_step/sec: 50.9929
INFO:tensorflow:loss = 1272.3849, step = 32701 (1.961 sec)
INFO:tensorflow:global_step/sec: 54.7204
INFO:tensorflow:loss = 465.31973, step = 32801 (1.827 sec)
INFO:tensorflow:global_step/sec: 56.9847
INFO:tens

INFO:tensorflow:global_step/sec: 55.4829
INFO:tensorflow:loss = 271.34863, step = 40201 (1.802 sec)
INFO:tensorflow:global_step/sec: 52.7594
INFO:tensorflow:loss = 205.65034, step = 40301 (1.896 sec)
INFO:tensorflow:global_step/sec: 49.3506
INFO:tensorflow:loss = 681.8507, step = 40401 (2.026 sec)
INFO:tensorflow:global_step/sec: 47.866
INFO:tensorflow:loss = 350.56168, step = 40501 (2.089 sec)
INFO:tensorflow:global_step/sec: 49.9059
INFO:tensorflow:loss = 730.4547, step = 40601 (2.004 sec)
INFO:tensorflow:global_step/sec: 48.5067
INFO:tensorflow:loss = 258.51477, step = 40701 (2.061 sec)
INFO:tensorflow:global_step/sec: 57.8974
INFO:tensorflow:loss = 141.95517, step = 40801 (1.727 sec)
INFO:tensorflow:global_step/sec: 51.3229
INFO:tensorflow:loss = 452.39703, step = 40901 (1.950 sec)
INFO:tensorflow:global_step/sec: 56.759
INFO:tensorflow:loss = 147.2377, step = 41001 (1.760 sec)
INFO:tensorflow:global_step/sec: 53.9066
INFO:tensorflow:loss = 1083.1736, step = 41101 (1.855 sec)
INFO:

INFO:tensorflow:loss = 1043.9923, step = 48401 (1.871 sec)
INFO:tensorflow:global_step/sec: 55.4213
INFO:tensorflow:loss = 280.69565, step = 48501 (1.804 sec)
INFO:tensorflow:global_step/sec: 51.3798
INFO:tensorflow:loss = 910.7396, step = 48601 (1.947 sec)
INFO:tensorflow:global_step/sec: 48.5134
INFO:tensorflow:loss = 799.23566, step = 48701 (2.061 sec)
INFO:tensorflow:global_step/sec: 52.01
INFO:tensorflow:loss = 113.52008, step = 48801 (1.923 sec)
INFO:tensorflow:global_step/sec: 48.5069
INFO:tensorflow:loss = 390.43906, step = 48901 (2.062 sec)
INFO:tensorflow:global_step/sec: 51.6366
INFO:tensorflow:loss = 270.44247, step = 49001 (1.936 sec)
INFO:tensorflow:global_step/sec: 51.0544
INFO:tensorflow:loss = 326.15427, step = 49101 (1.959 sec)
INFO:tensorflow:global_step/sec: 48.7245
INFO:tensorflow:loss = 159.42224, step = 49201 (2.052 sec)
INFO:tensorflow:global_step/sec: 48.583
INFO:tensorflow:loss = 199.03973, step = 49301 (2.059 sec)
INFO:tensorflow:global_step/sec: 48.5679
INFO